In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import torch.optim as optim

if torch.cuda.is_available():  
    torch.cuda.empty_cache()
    dev = "cuda:0" 
else:  
    dev = "cpu"

class ResNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, pretrained = True, freeze = False):
        super(ResNet, self).__init__()
        self.model = models.resnet18(pretrained=pretrained)
        self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

        if freeze:
            for param in self.model.parameters():
                param.requires_grad = False
            for param in self.model.fc.parameters():
                param.requires_grad = True


    def forward(self, x):
        return self.model(x)

#x = torch.randn(1,3,256,256)


In [ ]:
import csv

def SaveCsvForPlot(filename, train_result, test_result):
    data = [(train_acc, test_acc) for train_acc, test_acc in zip(train_result, test_result)]
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(data)

def train(model, train_set, test_set, batch_size, epochs, lr, csv_name = "report"):
    train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
    test_loader = DataLoader(test_set, batch_size = batch_size)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_accuracy_plot = []
    test_accuracy_plot = []

    for epoch in range(epochs):
        model.train()
        train_loss=0
        train_accuracy=0.0
        for x, y in tqdm(train_loader):
            x, y = x.to(dev).float(), y.to(dev).to(torch.int64)
            optimizer.zero_grad()
            output=model(x).float()
            y = y.squeeze()
            loss=loss_function(output,y)
            loss.backward()
            train_loss+=loss
            optimizer.step()
            _,preds = torch.max(output.data,1)
            train_accuracy+=int(torch.sum(preds==y.data))
            
        model.eval()
        test_loss=0
        test_accuracy=0.0
        with torch.no_grad():
            for x, y in tqdm(test_loader):
                x, y = x.to(dev).float(), y.to(dev).to(torch.int64)
                output=model(x).float()
                y = y.squeeze()
                loss=loss_function(output,y)
                test_loss+=loss
                _,preds = torch.max(output.data,1)
                test_accuracy+=int(torch.sum(preds==y.data))

        print(f"Epoch {epoch+1}: train accuracy = {train_accuracy/len(train_set)}, test accuracy = {test_accuracy/len(test_set)}")    
        train_accuracy_plot.append(train_accuracy/len(train_set))
        test_accuracy_plot.append(test_accuracy/len(test_set)) 
        SaveCsvForPlot(f'{csv_name}.csv',train_accuracy_plot, test_accuracy_plot)

In [ ]:
train_x, train_y, test_x, test_y =torch.from_numpy(np.load('CIFAR-10/x_train.npy')).permute(0, 3, 1, 2), torch.from_numpy(np.load('CIFAR-10/y_train.npy')), torch.from_numpy(np.load('CIFAR-10/x_test.npy')).permute(0, 3, 1, 2), torch.from_numpy(np.load('CIFAR-10/y_test.npy'))


In [ ]:
train_set = TensorDataset(train_x, train_y)
test_set = TensorDataset(test_x, test_y)

model = ResNet().to(dev)
batch_size = 16
epochs = 10
lr = 1e-4
csv_name = "Pretrained_ResNet18"

train(model, train_set, test_set, batch_size, epochs, lr, csv_name)


In [ ]:
train_set = TensorDataset(train_x, train_y)
test_set = TensorDataset(test_x, test_y)

model = ResNet(pretrained = False).to(dev)
batch_size = 16
epochs = 10
lr = 1e-4
csv_name = "ResNet18"

train(model, train_set, test_set, batch_size, epochs, lr, csv_name)

In [ ]:
train_set = TensorDataset(train_x, train_y)
test_set = TensorDataset(test_x, test_y)

model = ResNet(freeze = True).to(dev)
batch_size = 16
epochs = 10
lr = 1e-4
csv_name = "Freezed_ResNet18"

train(model, train_set, test_set, batch_size, epochs, lr, csv_name)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Pretrained = pd.read_csv('Pretrained_ResNet18.csv', header = None)
From_Scratch = pd.read_csv('ResNet18.csv', header = None)
Freezed = pd.read_csv("Freezed_ResNet18.csv", header = None)

train_column = 0  # Column index for train accuracy
valid_column = 1  # Column index for validation accuracy

# Plotting all models together
plt.figure(figsize=(10, 3))

# ResNet18
plt.plot(Pretrained[train_column], label='ResNet18 Train')
plt.plot(Pretrained[valid_column], label='ResNet18 Validation')

# ResNet50
plt.plot(From_Scratch[train_column], label='ResNet50 Train')
plt.plot(From_Scratch[valid_column], label='ResNet50 Validation')

# ResNet152
plt.plot(Freezed[train_column], label='ResNet152 Train')
plt.plot(Freezed[valid_column], label='ResNet152 Validation')

plt.title('Training and Validation Accuracy for ResNet Models')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()